# 4.ライブラリによる分析実験(67.5/100)


# 4-2.pandas(7:17.5)

## 1 Pandasの概要

### pandasとは

SeriesとDataFrameというデータ型を提供するためのパッケージ

In [ ]:
import pandas as pd

### Seriesとは
1次元データ

In [ ]:
ser = pd.Series([10,20,30,40])
ser
# 全ての要素を整数で作成したので、自動的にdtype:int64が割り当てられてる

0    10
1    20
2    30
3    40
dtype: int64

### DataFrameとは
2次元データ  
列毎にデータ型が統一されているなら、データの計算が容易に行える。  
複数種類のデータ型が混在している列は、データ型がObjectになる。  
Object型の列は計算できない。

In [ ]:
df = pd.DataFrame([[10,'a',True],
                   [20,'b',False],
                   [30,'c',False],
                   [40,'d',True]])
df

,0,1,2
0,10,a,True
1,20,b,False
2,30,c,False
3,40,d,True


### DataFrameの概要を見る
Numpyのarange関数を使ってデータを作ってみるよ

In [ ]:
import numpy as np
df = pd.DataFrame(np.arange(100).reshape((25,4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
6,24,25,26,27
7,28,29,30,31
8,32,33,34,35
9,36,37,38,39


In [ ]:
df.head() # 先頭5行を出力するよ

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [ ]:
df.tail() # 末尾5行を出力するよ

,0,1,2,3
20,80,81,82,83
21,84,85,86,87
22,88,89,90,91
23,92,93,94,95
24,96,97,98,99


In [ ]:
df.shape # 列数、行数を出力するよ

(25, 4)

### インデックス名、カラム名
わかりやすいインデックス名、カラム名をつけたいよね

In [ ]:
df = pd.DataFrame(np.arange(6).reshape((3,2)))
df

,0,1
0,0,1
1,2,3
2,4,5


In [ ]:
df.index = ['01','02','03']
df.columns = ['A','B']
df

,A,B
01,0,1
02,2,3
03,4,5


In [ ]:
named_df = pd.DataFrame(np.arange(6).reshape((3,2)),
                        columns=['A列','B列'],
                        index = ['1行目','2行目','3行目'])
named_df

,A列,B列
1行目,0,1
2行目,2,3
3行目,4,5


### データの抽出
データを投句って、データを抽出してみる。

In [ ]:
import numpy as np
import pandas as pd
df = pd.DataFrame(np.arange(12).reshape((4,3)),
                  columns = ['A','B','C'],
                  index = ['1行目','2行目','3行目','4行目'])
df

,A,B,C
1行目,0,1,2
2行目,3,4,5
3行目,6,7,8
4行目,9,10,11


In [ ]:
df['A'] # 列で抽出

1行目    0
2行目    3
3行目    6
4行目    9
Name: A, dtype: int64

In [ ]:
df[['A','C']] # リストで複数列抽出

,A,C
1行目,0,2
2行目,3,5
3行目,6,8
4行目,9,11


In [ ]:
df[:2] # 2行目まで抽出

,A,B,C
1行目,0,1,2
2行目,3,4,5


In [ ]:
df.loc[:,:] # locメソッドで全行全列抽出

,A,B,C
1行目,0,1,2
2行目,3,4,5
3行目,6,7,8
4行目,9,10,11


In [ ]:
df.loc[:,'A'] # locメソッドで全行A列抽出

1行目    0
2行目    3
3行目    6
4行目    9
Name: A, dtype: int64

In [ ]:
df.loc[:,['A','B']] # locメソッドで全行A列B列抽出

,A,B
1行目,0,1
2行目,3,4
3行目,6,7
4行目,9,10


In [ ]:
df.loc['1行目',:] # 1行目の全て抽出

A    0
B    1
C    2
Name: 1行目, dtype: int64

In [ ]:
df.loc[['1行目','3行目'],:] # 1行目、3行目の全て抽出

,A,B,C
1行目,0,1,2
3行目,6,7,8


In [ ]:
df.loc[['1行目'],['A','C']] # 1行目のA、C列を抽出

,A,C
1行目,0,2


In [ ]:
df.iloc[1,1] # 数字で抽出（この場合、2行目のB列）

4

In [ ]:
df.iloc[1:,1]

2行目     4
3行目     7
4行目    10
Name: B, dtype: int64

In [ ]:
df.iloc[1:,:2]

,A,B
2行目,3,4
3行目,6,7
4行目,9,10


## 2. データの読み込み・書き込み

### データ読み込み：CSVファイル

In [12]:
import pandas as pd
df = pd.read_csv('data/201704health.csv', encoding='utf-8')
df

,日付,歩数,摂取カロリー
0,2017-04-01,5439,2500
1,2017-04-02,2510,2300
2,2017-04-03,10238,1950
3,2017-04-04,8209,1850
4,2017-04-05,9434,1930
5,2017-04-06,7593,1800
6,2017-04-07,9320,1940
7,2017-04-08,4873,2300
8,2017-04-09,12045,1950
9,2017-04-10,7493,1850


### データ読み込み：Excelファイル

In [13]:
df = pd.read_excel('data/201704health.xlsx')
df

,日付,歩数,摂取カロリー
0,2017-04-01,5439,2500
1,2017-04-02,2510,2300
2,2017-04-03,10238,1950
3,2017-04-04,8209,1850
4,2017-04-05,9434,1930
5,2017-04-06,7593,1800
6,2017-04-07,9320,1940
7,2017-04-08,4873,2300
8,2017-04-09,12045,1950
9,2017-04-10,7493,1850


### データ読み込み：WebサイトのHTMLから表を取得

In [14]:
url = 'https://ja.wikipedia.org/wiki/%E3%83%88%E3%83%83%E3%83%97%E3%83%AC%E3%83%99%E3%83%AB%E3%83%89%E3%83%A1%E3%82%A4%E3%83%B3%E4%B8%80%E8%A6%A7'
tables = pd.read_html(url)

In [15]:
len(tables)

41

In [20]:
df = tables[4]
df

,Name,Entity,Explanation,Notes,IDN,DNSSEC,SLD,IPv6
0,.ac,アセンション島,NaN,広く学術的なサイト（大学など）のために用いられる。アセンション島はイギリス領であるが、イギリ...,Yes,Yes,Yes,Yes
1,.ad,アンドラ,NaN,アンドラにおける商標または市民権が必要となる[7][8]。,NaN,Yes,Yes,NaN
2,.ae,アラブ首長国連邦,NaN,NaN,NaN,No,Yes,NaN
3,.af,アフガニスタン,NaN,NaN,NaN,Yes,Yes,NaN
4,.ag,アンティグア・バーブーダ,NaN,AGがドイツの株式会社(Aktiengesellschaft)の略称であることから、非公式に...,NaN,Yes,Yes,NaN
...,...,...,...,...,...,...,...,...
248,.ye,イエメン,NaN,NaN,NaN,No,No,NaN
249,.yt,マヨット島,NaN,欧州連合・スイス・ノルウェー・アイスランド・リヒテンシュタインの個人・企業に限る[cctld...,Yes[cctld 12],Yes,Yes,NaN
250,.za,南アフリカ,Zuid-Afrika (オランダ語),NaN,NaN,No,No,NaN
251,.zm,ザンビア,NaN,NaN,NaN,Yes,Yes[cctld 28],NaN


### データ書き込み：CSVファイル

In [24]:
df.to_csv('data/write_data.csv')

TypeError: 'str' object is not callable

### データ書き込み：Excelファイル

In [23]:
df.to_excel('data/write_data.xlsx')

### データの再利用

In [25]:
df.to_pickle('data/write_df.pickle')

In [26]:
df = pd.read_pickle('data/write_df.pickle')

## 3.データの整形

In [27]:
import pandas as pd
import numpy as np

### 使用するデータの読み込み

In [28]:
df = pd.read_excel('data/201704health.xlsx')
df

,日付,歩数,摂取カロリー
0,2017-04-01,5439,2500
1,2017-04-02,2510,2300
2,2017-04-03,10238,1950
3,2017-04-04,8209,1850
4,2017-04-05,9434,1930
5,2017-04-06,7593,1800
6,2017-04-07,9320,1940
7,2017-04-08,4873,2300
8,2017-04-09,12045,1950
9,2017-04-10,7493,1850


### 条件で抽出

In [29]:
df['歩数'] >= 10000

0     False
1     False
2      True
3     False
4     False
5     False
6     False
7     False
8      True
9     False
10    False
11    False
12     True
13    False
14    False
15    False
16    False
17    False
18    False
19     True
20     True
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
Name: 歩数, dtype: bool

In [31]:
df_selected = df[df['歩数'] >= 10000]
df_selected

,日付,歩数,摂取カロリー
2,2017-04-03,10238,1950
8,2017-04-09,12045,1950
12,2017-04-13,10287,1800
19,2017-04-20,15328,1800
20,2017-04-21,12849,1940


In [32]:
df_selected.shape

(5, 3)

In [33]:
df.query('歩数 >= 10000 and 摂取カロリー <= 1800')

,日付,歩数,摂取カロリー
12,2017-04-13,10287,1800
19,2017-04-20,15328,1800


### データ型変換

In [34]:
df.dtypes

日付        object
歩数         int64
摂取カロリー     int64
dtype: object

In [35]:
df.loc[:, 'date'] = df.loc[:,'日付'].apply(pd.to_datetime)

In [36]:
df.loc[:,'date']

0    2017-04-01
1    2017-04-02
2    2017-04-03
3    2017-04-04
4    2017-04-05
5    2017-04-06
6    2017-04-07
7    2017-04-08
8    2017-04-09
9    2017-04-10
10   2017-04-11
11   2017-04-12
12   2017-04-13
13   2017-04-14
14   2017-04-15
15   2017-04-16
16   2017-04-17
17   2017-04-18
18   2017-04-19
19   2017-04-20
20   2017-04-21
21   2017-04-22
22   2017-04-23
23   2017-04-24
24   2017-04-25
25   2017-04-26
26   2017-04-27
27   2017-04-28
28   2017-04-29
29   2017-04-30
Name: date, dtype: datetime64[ns]

In [37]:
df

,日付,歩数,摂取カロリー,date
0,2017-04-01,5439,2500,2017-04-01
1,2017-04-02,2510,2300,2017-04-02
2,2017-04-03,10238,1950,2017-04-03
3,2017-04-04,8209,1850,2017-04-04
4,2017-04-05,9434,1930,2017-04-05
5,2017-04-06,7593,1800,2017-04-06
6,2017-04-07,9320,1940,2017-04-07
7,2017-04-08,4873,2300,2017-04-08
8,2017-04-09,12045,1950,2017-04-09
9,2017-04-10,7493,1850,2017-04-10


In [38]:
df.loc[:,'摂取カロリー'] = df.loc[:,'摂取カロリー'].astype(np.float32)

In [39]:
df = df.set_index('date')

In [40]:
df.head()

,日付,歩数,摂取カロリー
date,,,
2017-04-01,2017-04-01,5439,2500.0
2017-04-02,2017-04-02,2510,2300.0
2017-04-03,2017-04-03,10238,1950.0
2017-04-04,2017-04-04,8209,1850.0
2017-04-05,2017-04-05,9434,1930.0


### 並べ替え

In [41]:
df.sort_values(by='歩数')

,日付,歩数,摂取カロリー
date,,,
2017-04-02,2017-04-02,2510,2300.0
2017-04-23,2017-04-23,3890,1950.0
2017-04-22,2017-04-22,4029,2300.0
2017-04-30,2017-04-30,4093,1950.0
2017-04-08,2017-04-08,4873,2300.0
2017-04-01,2017-04-01,5439,2500.0
2017-04-29,2017-04-29,6033,2300.0
2017-04-12,2017-04-12,6481,2300.0
2017-04-27,2017-04-27,7203,1930.0


In [42]:
df.sort_values(by='歩数', ascending=False).head()

,日付,歩数,摂取カロリー
date,,,
2017-04-20,2017-04-20,15328,1800.0
2017-04-21,2017-04-21,12849,1940.0
2017-04-09,2017-04-09,12045,1950.0
2017-04-13,2017-04-13,10287,1800.0
2017-04-03,2017-04-03,10238,1950.0


### 不要なカラムの削除

In [43]:
df = df.drop('日付',axis=1)

In [44]:
df.tail()

,歩数,摂取カロリー
date,,
2017-04-26,7492,1850.0
2017-04-27,7203,1930.0
2017-04-28,7302,1850.0
2017-04-29,6033,2300.0
2017-04-30,4093,1950.0


### 組合わせデータの挿入

In [45]:
df.loc[:,'歩数/カロリー'] = df.loc[:,'歩数'] / df.loc[:,'摂取カロリー']
df

,歩数,摂取カロリー,歩数/カロリー
date,,,
2017-04-01,5439,2500.0,2.175600
2017-04-02,2510,2300.0,1.091304
2017-04-03,10238,1950.0,5.250256
2017-04-04,8209,1850.0,4.437297
2017-04-05,9434,1930.0,4.888083
2017-04-06,7593,1800.0,4.218333
2017-04-07,9320,1940.0,4.804124
2017-04-08,4873,2300.0,2.118696
2017-04-09,12045,1950.0,6.176923


In [46]:
def exercise_judge(ex):
    if ex <= 3.0:
        return 'Low'
    elif 3.0 < ex <= 6.0:
        return 'Mid'
    else:
        return 'High'

In [47]:
df.loc[:,'運動指数'] = df.loc[:,'歩数/カロリー'].apply(exercise_judge)
df

,歩数,摂取カロリー,歩数/カロリー,運動指数
date,,,,
2017-04-01,5439,2500.0,2.175600,Low
2017-04-02,2510,2300.0,1.091304,Low
2017-04-03,10238,1950.0,5.250256,Mid
2017-04-04,8209,1850.0,4.437297,Mid
2017-04-05,9434,1930.0,4.888083,Mid
2017-04-06,7593,1800.0,4.218333,Mid
2017-04-07,9320,1940.0,4.804124,Mid
2017-04-08,4873,2300.0,2.118696,Low
2017-04-09,12045,1950.0,6.176923,High


In [48]:
df.to_pickle('data/df_201704health.pickle')

In [49]:
df_moved = pd.get_dummies(df.loc[:,'運動指数'], prefix='運動')
df_moved

,運動_High,運動_Low,運動_Mid
date,,,
2017-04-01,0,1,0
2017-04-02,0,1,0
2017-04-03,0,0,1
2017-04-04,0,0,1
2017-04-05,0,0,1
2017-04-06,0,0,1
2017-04-07,0,0,1
2017-04-08,0,1,0
2017-04-09,1,0,0


In [50]:
df_moved.to_pickle('data/df_201704moved.pickle')

## 4.時系列データ

### 1ヶ月分のデータを作る

### 1年分365日のデータを作る

### 月平均のデータにする

### 複雑な条件のインデックス

## 5.欠損値処理

## 6.データ連結

### 保存したデータを読み込み

### 列方向のデータ連結

### 行方向のデータ連結

## 7.統計データの扱い

### 保存したデータの読み込み

### 基本統計量

### 要約

### 相関係数

### 散布図行列

### データ変換